# Scheduled Integration of ClinGen Gene-Disease Validity Data into WikiData

ClinGen (Clinical Genome Resource) develops curated data of genetic associations <br>
CC0 https://clinicalgenome.org/docs/terms-of-use/

This scheduled bot operates through WDI to integrate ClinGen Gene-Disease Validity Data <br>
http://jenkins.sulab.org/ <br>
https://github.com/SuLab/GeneWikiCentral/issues/116 <br>
https://search.clinicalgenome.org/kb/gene-validity/ <br>

Python script contributions, in order: Sabah Ul-Hasan, Andra Waagmeester, Andrew Su, Ginger Tsueng

## Checks

- Login should automatically align with given environment 
- For loop checks for both HGNC Qid and MONDO Qid per each row (ie if HGNC absent or multiple, then checks MONDO) 
- For loop works on multiple Qid option, tested for A2ML1 and corrected afterwards
- For loop puts correct Qid for either HGNC or MONDO, if available 

## Issues

- Why does output say a row is a complete, when this is not true in WikiData using scheduled bot?
- create_reference() and update_retrieved_if_new_multiple_refs functions adds and/or updates ref to existing HGNC or MONDO value in genetic association statement within 180 days (doesn't overwrite URLs from non-ClinGen sources)
- Updated, not updated, skipped - not definitive or mapping error..
- Maybe get of Definitive column, but keep Gene and Disease QID


### Relevant modules and libraries

In [4]:
# Installations by shell 
!pip install --upgrade pip # Installs pip, ensures it's up-to-date
!pip3 install tqdm # Visualizes installation progress (progress bar)
!pip3 install termcolor # For color-coding printed output
!pip3 install wikidataintegrator # For wikidata

Requirement already up-to-date: pip in /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages (19.3.1)


In [2]:
# Installations by python
from wikidataintegrator import wdi_core, wdi_login 
from wikidataintegrator.ref_handlers import update_retrieved_if_new_multiple_refs
from datetime import datetime
#from termcolor import colored 

import pandas as pd
import numpy as np

#import ssl
#ssl._create_default_https_context = ssl._create_unverified_context

import os
import copy 
import time 

### Login for running WDI

### ClinGen gene-disease validity data

In [3]:
print("Logging in...") 

# Enter your own username and password ** to be updated to ProteinBoxBot
os.environ["WDUSER"] = "username" # Uses os package to call and set the environment for wikidata username
os.environ["WDPASS"] = "password"

# Conditional that outputs error command if not in the local python environment
if "WDUSER" in os.environ and "WDPASS" in os.environ: 
    WDUSER = os.environ['WDUSER']
    WDPASS = os.environ['WDPASS']
else: 
    raise ValueError("WDUSER and WDPASS must be specified in local.py or as environment variables")      

# Sets attributed username and password as 'login'
login = wdi_login.WDLogin(WDUSER, WDPASS) 

Logging in...
https://www.wikidata.org/w/api.php
Successfully logged in as Sulhasan


In [4]:
# Read as csv
df = pd.read_csv('https://search.clinicalgenome.org/kb/gene-validity.csv', skiprows=6, header=None)  

# Label column headings
df.columns = ['Gene', 'HGNC Gene ID', 'Disease', 'MONDO Disease ID','SOP','Classification','Report Reference URL','Report Date']

# Create time stamp of when downloaded (error if isoformat() used)
timeStringNow = datetime.now().strftime("+%Y-%m-%dT00:00:00Z")

df.head(6) # View first 6 rows

,Gene,HGNC Gene ID,Disease,MONDO Disease ID,SOP,Classification,Report Reference URL,Report Date
0,A2ML1,HGNC:23336,Noonan syndrome with multiple lentigines,MONDO_0007893,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:37:47.175Z
1,A2ML1,HGNC:23336,cardiofaciocutaneous syndrome,MONDO_0015280,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:31:03.696Z
2,A2ML1,HGNC:23336,Costello syndrome,MONDO_0009026,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:34:05.324Z
3,A2ML1,HGNC:23336,Noonan syndrome,MONDO_0018997,SOP5,Disputed,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:23:53.157Z
4,A2ML1,HGNC:23336,Noonan syndrome-like disorder with loose anage...,MONDO_0011899,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:40:11.599Z
5,AARS,HGNC:20,undetermined early-onset epileptic encephalopathy,MONDO_0018614,SOP6,Limited,https://search.clinicalgenome.org/kb/gene-vali...,2018-11-20T17:00:00.000Z


In [5]:
### Create empty columns for output file (ignore warnings)

df['Status'] = "pending" # "Status" column with 'pending' for all cells: 'error', 'complete', 'skipped' (meaning previously logged within 180 days)
df['Definitive'] = "" # Empty cell to be replaced with 'yes' or 'no' string
df['Gene QID'] = "" # To be replaced with 'absent' or 'multiple'
df['Disease QID'] = "" # To be replaced with 'absent' or 'multiple'

df.head(6)

,Gene,HGNC Gene ID,Disease,MONDO Disease ID,SOP,Classification,Report Reference URL,Report Date,Status,Definitive,Gene QID,Disease QID
0,A2ML1,HGNC:23336,Noonan syndrome with multiple lentigines,MONDO_0007893,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:37:47.175Z,pending,,,
1,A2ML1,HGNC:23336,cardiofaciocutaneous syndrome,MONDO_0015280,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:31:03.696Z,pending,,,
2,A2ML1,HGNC:23336,Costello syndrome,MONDO_0009026,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:34:05.324Z,pending,,,
3,A2ML1,HGNC:23336,Noonan syndrome,MONDO_0018997,SOP5,Disputed,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:23:53.157Z,pending,,,
4,A2ML1,HGNC:23336,Noonan syndrome-like disorder with loose anage...,MONDO_0011899,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:40:11.599Z,pending,,,
5,AARS,HGNC:20,undetermined early-onset epileptic encephalopathy,MONDO_0018614,SOP6,Limited,https://search.clinicalgenome.org/kb/gene-vali...,2018-11-20T17:00:00.000Z,pending,,,


In [6]:
### Create a function for adding references to then be iterated in the loop "create_reference()"

def create_reference(): # Indicates a parameter included before running rest of function (otherwise may not recognize)
        refStatedIn = wdi_core.WDItemID(value="Q64403342", prop_nr="P248", is_reference=True) # ClinGen Qid = Q64403342, 'stated in' Pid = P248 
        timeStringNow = datetime.now().strftime("+%Y-%m-%dT00:00:00Z") # Create time stamp of when downloaded (error if isoformat() used)
        refRetrieved = wdi_core.WDTime(timeStringNow, prop_nr="P813", is_reference=True) # Calls on previous 'timeStringNow' string, 'retrieved' Pid = P813
        refURL = wdi_core.WDUrl((df.loc[index, 'Report Reference URL']), prop_nr="P854", is_reference=True) # 'reference URL' Pid = P854
        return [refStatedIn, refRetrieved, refURL]

### For loop that executes the following through each row of the dataframe 

In [7]:
start_time = time.time() # Keep track of how long it takes loop to run

for index, row in df.iterrows(): # Index is a row number, row is all variables and values for that row
        
    # Identify the string in the Gene or Disease column for a given row
    HGNC = df.loc[index, 'HGNC Gene ID'].replace("HGNC:", "") # .replace() changes HGNC: to space for SparQL query
    MONDO = df.loc[index, 'MONDO Disease ID'].replace("_", ":")
    
    # SparQL query to search for Gene or Disease in Wikidata based on HGNC ID (P354) or MonDO ID (P5270)
    sparqlQuery_HGNC = "SELECT * WHERE {?gene wdt:P354 \""+HGNC+"\"}" 
    result_HGNC = wdi_core.WDItemEngine.execute_sparql_query(sparqlQuery_HGNC) # Resultant query
    sparqlQuery_MONDO = "SELECT * WHERE {?disease wdt:P5270 \""+MONDO+"\"}" 
    result_MONDO = wdi_core.WDItemEngine.execute_sparql_query(sparqlQuery_MONDO)
    
    # Assign resultant length of dictionary for either Gene or Disease (number of Qid)
    HGNC_qlength = len(result_HGNC["results"]["bindings"]) 
    MONDO_qlength = len(result_MONDO["results"]["bindings"])
    
    # Conditional utilizing length value for output table, accounts for absent/present combos
    if HGNC_qlength == 1:
        HGNC_qid = result_HGNC["results"]["bindings"][0]["gene"]["value"].replace("http://www.wikidata.org/entity/", "")
        df.at[index, 'Gene QID'] = HGNC_qid # Input HGNC Qid in 'Gene QID' cell  
    if HGNC_qlength < 1: # If no Qid
        df.at[index, 'Status'] = "error" 
        df.at[index, 'Gene QID'] = "absent"  
    if HGNC_qlength > 1: # If multiple Qid
        df.at[index, 'Status'] = "error" 
        df.at[index, 'Gene QID'] = "multiple"
        
    if MONDO_qlength == 1:
        MONDO_qid = result_MONDO["results"]["bindings"][0]["disease"]["value"].replace("http://www.wikidata.org/entity/", "") 
        df.at[index, 'Disease QID'] = MONDO_qid  
    if MONDO_qlength < 1: 
        df.at[index, 'Status'] = "error" 
        df.at[index, 'Disease QID'] = "absent" 
    if MONDO_qlength > 1:
        df.at[index, 'Status'] = "error" 
        df.at[index, 'Disease QID'] = "multiple" 
        
    # Conditional inputs error such that only rows are written for where Classification = 'Definitive'
    if row['Classification']!='Definitive': # If the string is NOT 'Definitive' for the Classification column
        df.at[index, 'Status'] = "error" # Then input "error" in the Status column
        df.at[index, 'Definitive'] = "no" # And'no' for Definitive column
        continue # Skips rest and goes to next row
    else: # Otherwise
        df.at[index, 'Definitive'] = "yes" # Input 'yes' for Definitive column, go to next step
  
    # Conditional continues to write into WikiData only if 1 Qid for each + Definitive classification 
    if HGNC_qlength == 1 & MONDO_qlength == 1:
        
        # Call upon create_reference() function created   
        reference = create_reference() 
        
        # Add disease value to gene item page, and gene value to disease item page (symmetry)
        
        # Creates 'gene assocation' statement (P2293) whether or not it's already there, and includes the references
        statement_HGNC = [wdi_core.WDItemID(value=MONDO_qid, prop_nr="P2293", references=[copy.deepcopy(reference)])] 
        wikidata_HGNCitem = wdi_core.WDItemEngine(wd_item_id=HGNC_qid, 
                                                  data=statement_HGNC, 
                                                  global_ref_mode='CUSTOM', # parameter that looks within 180 days
                                                  ref_handler=update_retrieved_if_new_multiple_refs, 
                                                  append_value=["P2293"])
        wikidata_HGNCitem.get_wd_json_representation() # Gives json structure that submitted to API, helpful for debugging 
        
        statement_MONDO = [wdi_core.WDItemID(value=HGNC_qid, prop_nr="P2293", references=[copy.deepcopy(reference)])] 
        wikidata_MONDOitem = wdi_core.WDItemEngine(wd_item_id=MONDO_qid, 
                                                   data=statement_MONDO, 
                                                   global_ref_mode='CUSTOM',
                                                   ref_handler=update_retrieved_if_new_multiple_refs, 
                                                   append_value=["P2293"])
        wikidata_MONDOitem.get_wd_json_representation()
        
        # Write message for combination successfully logged, and enter 'complete' in Status column
        HGNC_name = df.loc[index, 'Gene'] # To output gene name > HGNC ID
        MONDO_name = df.loc[index, 'Disease']
        df.at[index, 'Status'] = "complete" 
        
        #print(colored(HGNC_name,"blue"), "Gene with HGNC ID", 
        #      colored(HGNC,"blue"), "logged as Qid",
        #      colored(wikidata_HGNCitem.write(login),"blue"),
        #      "and")
        #print(colored(MONDO_name,"green"), "Disease with MONDO ID", 
        #      colored(MONDO,"green"), "logged as Qid",
        #      colored(wikidata_MONDOitem.write(login),"green"))
        
        
end_time = time.time() # Captures when loop run ends
print("The total time of this loop is:", end_time - start_time, "seconds, or", (end_time - start_time)/60, "minutes")

# Write output to a .csv file
now = datetime.now() # Retrieves current time and saves it as 'now'
# Includes hour:minute:second_dd-mm-yyyy time stamp (https://en.wikipedia.org/wiki/ISO_8601)
df.to_csv("ClinGenBot_Status-Output_" + now.isoformat() + ".csv")  # isoformat
df

ABCC9 Gene with HGNC ID 60 logged as Qid Q18034993 and
hypertrichotic osteochondrodysplasia Cantu type Disease with MONDO ID MONDO:0009406 logged as Qid Q5034093
ABCD1 Gene with HGNC ID 61 logged as Qid Q14912808 and
X-linked cerebral adrenoleukodystrophy Disease with MONDO ID MONDO:0010247 logged as Qid Q55345732
ABHD12 Gene with HGNC ID 15868 logged as Qid Q18038087 and
PHARC syndrome Disease with MONDO ID MONDO:0012984 logged as Qid Q32137273
ACAD8 Gene with HGNC ID 87 logged as Qid Q18038564 and
isobutyryl-CoA dehydrogenase deficiency Disease with MONDO ID MONDO:0012648 logged as Qid Q6085391
ACAD9 Gene with HGNC ID 21497 logged as Qid Q18039534 and
acyl-CoA dehydrogenase 9 deficiency Disease with MONDO ID MONDO:0012624 logged as Qid Q55783804
ACADM Gene with HGNC ID 89 logged as Qid Q4650495 and
medium chain acyl-CoA dehydrogenase deficiency Disease with MONDO ID MONDO:0008721 logged as Qid Q750826
ACADS Gene with HGNC ID 90 logged as Qid Q4650490 and
short chain acyl-CoA dehydrog

BRIP1 Gene with HGNC ID 20473 logged as Qid Q18047355 and
Fanconi anemia complementation group j Disease with MONDO ID MONDO:0012187 logged as Qid Q32147098
BRWD3 Gene with HGNC ID 17342 logged as Qid Q18053726 and
X-linked syndromic intellectual disability Disease with MONDO ID MONDO:0020119 logged as Qid Q8041560
BSND Gene with HGNC ID 16512 logged as Qid Q18032616 and
Bartter disease type 4a Disease with MONDO ID MONDO:0011242 logged as Qid Q27674850
BTD Gene with HGNC ID 1122 logged as Qid Q18021004 and
biotinidase deficiency Disease with MONDO ID MONDO:0009665 logged as Qid Q776026
BUB1B Gene with HGNC ID 1149 logged as Qid Q17853927 and
mosaic variegated aneuploidy syndrome 1 Disease with MONDO ID MONDO:0009759 logged as Qid Q26492834
CA5A Gene with HGNC ID 1377 logged as Qid Q17854478 and
hyperammonemic encephalopathy due to carbonic anhydrase VA deficiency Disease with MONDO ID MONDO:0014332 logged as Qid Q55784783
CASK Gene with HGNC ID 1497 logged as Qid Q18033122 and
X-linke

exostoses, multiple, type 1 Disease with MONDO ID MONDO:0007585 logged as Qid Q55950215
EXT2 Gene with HGNC ID 3513 logged as Qid Q17917699 and
exostoses, multiple, type 2 Disease with MONDO ID MONDO:0007586 logged as Qid Q55950216
EYA1 Gene with HGNC ID 3519 logged as Qid Q17917742 and
branchio-oto-renal syndrome Disease with MONDO ID MONDO:0007029 logged as Qid Q2280106
EYA4 Gene with HGNC ID 3522 logged as Qid Q17917387 and
nonsyndromic genetic deafness Disease with MONDO ID MONDO:0019497 logged as Qid Q9079046
F5 Gene with HGNC ID 3542 logged as Qid Q14865116 and
thrombophilia due to activated protein C resistance Disease with MONDO ID MONDO:0008560 logged as Qid Q296104
FANCC Gene with HGNC ID 3584 logged as Qid Q18250517 and
Fanconi anemia complementation group C Disease with MONDO ID MONDO:0009213 logged as Qid Q32146953
FANCG Gene with HGNC ID 3588 logged as Qid Q17927524 and
Fanconi anemia complementation group G Disease with MONDO ID MONDO:0013565 logged as Qid Q32146937
FGD1

IVD Gene with HGNC ID 6186 logged as Qid Q3195538 and
isovaleric acidemia Disease with MONDO ID MONDO:0009475 logged as Qid Q3278042
KCNQ2 Gene with HGNC ID 6296 logged as Qid Q14914307 and
undetermined early-onset epileptic encephalopathy Disease with MONDO ID MONDO:0018614 logged as Qid Q56014174
KCNQ4 Gene with HGNC ID 6298 logged as Qid Q18033856 and
nonsyndromic genetic deafness Disease with MONDO ID MONDO:0019497 logged as Qid Q9079046
KDM5C Gene with HGNC ID 11114 logged as Qid Q18032784 and
X-linked syndromic intellectual disability Disease with MONDO ID MONDO:0020119 logged as Qid Q8041560
KLHL40 Gene with HGNC ID 30372 logged as Qid Q18050101 and
nemaline myopathy 8 Disease with MONDO ID MONDO:0014138 logged as Qid Q32144863
L1CAM Gene with HGNC ID 6470 logged as Qid Q14865255 and
L1 syndrome Disease with MONDO ID MONDO:0017140 logged as Qid Q56013931
LAMP2 Gene with HGNC ID 6501 logged as Qid Q18028468 and
Danon disease Disease with MONDO ID MONDO:0010281 logged as Qid Q5220

autosomal recessive nonsyndromic deafness 9 Disease with MONDO ID MONDO:0010986 logged as Qid Q28024662
OTOG Gene with HGNC ID 8516 logged as Qid Q18055453 and
nonsyndromic genetic deafness Disease with MONDO ID MONDO:0019497 logged as Qid Q9079046
OTOGL Gene with HGNC ID 26901 logged as Qid Q18054103 and
nonsyndromic genetic deafness Disease with MONDO ID MONDO:0019497 logged as Qid Q9079046
PAK3 Gene with HGNC ID 8592 logged as Qid Q18030342 and
X-linked syndromic intellectual disability Disease with MONDO ID MONDO:0020119 logged as Qid Q8041560
PALB2 Gene with HGNC ID 26144 logged as Qid Q18046302 and
Fanconi anemia complementation group N Disease with MONDO ID MONDO:0012565 logged as Qid Q32147054
PCDH19 Gene with HGNC ID 14270 logged as Qid Q18043114 and
epilepsy Disease with MONDO ID MONDO:0005027 logged as Qid Q41571
PHF6 Gene with HGNC ID 18145 logged as Qid Q18047542 and
Borjeson-Forssman-Lehmann syndrome Disease with MONDO ID MONDO:0010537 logged as Qid Q1019940
PHF8 Gene wit

hereditary pheochromocytoma-paraganglioma Disease with MONDO ID MONDO:0017366 logged as Qid Q56013982
SHANK3 Gene with HGNC ID 14294 logged as Qid Q18048003 and
Phelan McDermid syndrome Disease with MONDO ID MONDO:0011652 logged as Qid Q1926345
SHOC2 Gene with HGNC ID 15454 logged as Qid Q18032712 and
Noonan syndrome-like disorder with loose anagen hair Disease with MONDO ID MONDO:0011899 logged as Qid Q55783530
SIX1 Gene with HGNC ID 10887 logged as Qid Q18031509 and
branchio-oto-renal syndrome Disease with MONDO ID MONDO:0007029 logged as Qid Q2280106
SLC16A2 Gene with HGNC ID 10923 logged as Qid Q18031570 and
Allan-Herndon-Dudley syndrome Disease with MONDO ID MONDO:0010354 logged as Qid Q4731121
SLC22A5 Gene with HGNC ID 10969 logged as Qid Q14912759 and
systemic primary carnitine deficiency disease Disease with MONDO ID MONDO:0008919 logged as Qid Q3358135
SLC25A20 Gene with HGNC ID 1421 logged as Qid Q17855239 and
carnitine-acylcarnitine translocase deficiency Disease with MONDO 

UBE2A Gene with HGNC ID 12472 logged as Qid Q18032252 and
syndromic X-linked intellectual disability Nascimento type Disease with MONDO ID MONDO:0010461 logged as Qid Q28065624
UBE3A Gene with HGNC ID 12496 logged as Qid Q14878336 and
Angelman syndrome Disease with MONDO ID MONDO:0007113 logged as Qid Q535364
VHL Gene with HGNC ID 12687 logged as Qid Q14905473 and
hereditary pheochromocytoma-paraganglioma Disease with MONDO ID MONDO:0017366 logged as Qid Q56013982
VPS13B Gene with HGNC ID 2183 logged as Qid Q7907439 and
Cohen syndrome Disease with MONDO ID MONDO:0008999 logged as Qid Q1107087
WAS Gene with HGNC ID 12731 logged as Qid Q14863971 and
Wiskott-Aldrich syndrome Disease with MONDO ID MONDO:0010518 logged as Qid Q953638
WFS1 Gene with HGNC ID 12762 logged as Qid Q18032353 and
Wolfram syndrome Disease with MONDO ID MONDO:0018105 logged as Qid Q1153641
WFS1 Gene with HGNC ID 12762 logged as Qid Q18032353 and
Wolfram-like syndrome Disease with MONDO ID MONDO:0013673 logged as Qid

,Gene,HGNC Gene ID,Disease,MONDO Disease ID,SOP,Classification,Report Reference URL,Report Date,Status,Definitive,Gene QID,Disease QID
0,A2ML1,HGNC:23336,Noonan syndrome with multiple lentigines,MONDO_0007893,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:37:47.175Z,error,no,Q18051234,absent
1,A2ML1,HGNC:23336,cardiofaciocutaneous syndrome,MONDO_0015280,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:31:03.696Z,error,no,Q18051234,absent
2,A2ML1,HGNC:23336,Costello syndrome,MONDO_0009026,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:34:05.324Z,error,no,Q18051234,Q1136492
3,A2ML1,HGNC:23336,Noonan syndrome,MONDO_0018997,SOP5,Disputed,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:23:53.157Z,error,no,Q18051234,absent
4,A2ML1,HGNC:23336,Noonan syndrome-like disorder with loose anage...,MONDO_0011899,SOP5,No Reported Evidence,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-07T14:40:11.599Z,error,no,Q18051234,Q55783530
5,AARS,HGNC:20,undetermined early-onset epileptic encephalopathy,MONDO_0018614,SOP6,Limited,https://search.clinicalgenome.org/kb/gene-vali...,2018-11-20T17:00:00.000Z,error,no,Q17707208,Q56014174
6,AASS,HGNC:17366,hyperlysinemia (disease),MONDO_0009388,SOP6,Moderate,https://search.clinicalgenome.org/kb/gene-vali...,2019-11-08T17:00:00.000Z,error,no,Q18035079,absent
7,ABCC9,HGNC:60,hypertrichotic osteochondrodysplasia Cantu type,MONDO_0009406,SOP4,Definitive,https://search.clinicalgenome.org/kb/gene-vali...,2017-09-27T00:00:00,complete,yes,Q18034993,Q5034093
8,ABCD1,HGNC:61,X-linked cerebral adrenoleukodystrophy,MONDO_0010247,SOP4,Definitive,https://search.clinicalgenome.org/kb/gene-vali...,2018-02-07T14:00:00,complete,yes,Q14912808,Q55345732
9,ABHD12,HGNC:15868,PHARC syndrome,MONDO_0012984,SOP5,Definitive,https://search.clinicalgenome.org/kb/gene-vali...,2018-06-28T16:45:15.791Z,complete,yes,Q18038087,Q32137273
